# Setup

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams
import seaborn as sns
from pathlib import Path
import time
import sys
from IPython.display import clear_output

from fastprogress import master_bar, progress_bar

plt.rcParams['figure.figsize'] = (15,6)
plt.rcParams['font.size'] = 12
sns.set_theme(style='darkgrid')

In [2]:
import sys
sys.path.append('/home/ah19/runtime-monitoring/utilities')

In [3]:
from utils import *
from pathManager import fetchPaths

In [10]:
base = Path("/home/ah19/runtime-monitoring")
paths = fetchPaths(base, 'MNIST')

path = paths['mnist']
path_lastHiddenLayer = paths['lastHiddenLayer']

base, path

(PosixPath('/home/ah19/runtime-monitoring'),
 PosixPath('/home/ah19/runtime-monitoring/MNIST'))

# Import Data

In [9]:
model_name = 'MNIST-ElasticAllClasses'

df_val = pd.read_csv(path_lastHiddenLayer / f'{model_name}_val.csv')

df_val['true'] = (df_val['y_pred'] == df_val['y_true'])
df_val.rename({'y_true': 'y'}, axis=1, inplace=True)
df_val.drop(['y_pred'], axis=1, inplace=True)

print(df_val.shape)

df_val['true'].sum() / df_val.shape[0]

(6000, 32)


0.9505

# Bits

In [11]:
dft = (df_val.iloc[:, :-2] > 0).astype('int8')
dft += (df_val.iloc[:, :-2] > 0).astype('int8')

In [12]:
ttt = dft.head().to_numpy()
ttt.shape

(5, 30)

In [14]:
def multi_thlds(x):
    x = x.reshape(x.shape[0], 1)
    x = np.where(x == [0], [0, 0], # 0
         np.where(x == [1], [1, 0], # 1
         np.where(x == [2], [0, 1], # 2
         [1, 1] ) ) )# 3
    return np.reshape(x, x.shape[0] * 2)

In [15]:
ttt = np.apply_along_axis(multi_thlds, 1, ttt)

In [16]:
ttt = np.unique(ttt, axis=0)

In [17]:
ttt.shape

(5, 60)

In [18]:
ttt

array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 0, 

In [23]:
def add_one_pattern(x):
    # replace 1 with vars and 0 with vars_not
    return np.where( x == 1, 7, -3 )
#     expr = np.bitwise_and.reduce( expr )
#     self.roots |= expr

In [25]:
ttt = np.apply_along_axis(add_one_pattern, 1, ttt)
ttt

In [32]:
ttt[:, 0:2] = 1000

In [33]:
ttt

array([[1000, 1000,   -3,   -3,   -3,   -3,   -3,    7,   -3,   -3,   -3,
          -3,   -3,   -3,   -3,    7,   -3,    7,   -3,   -3,   -3,    7,
          -3,   -3,   -3,   -3,   -3,   -3,   -3,   -3,   -3,    7,   -3,
           7,   -3,    7,   -3,   -3,   -3,    7,   -3,   -3,   -3,   -3,
          -3,   -3,   -3,   -3,   -3,   -3,   -3,    7,   -3,    7,   -3,
          -3,   -3,   -3,   -3,   -3],
       [1000, 1000,   -3,   -3,   -3,   -3,   -3,    7,   -3,   -3,   -3,
           7,   -3,   -3,   -3,    7,   -3,    7,   -3,   -3,   -3,    7,
          -3,   -3,   -3,   -3,   -3,    7,   -3,   -3,   -3,    7,   -3,
           7,   -3,    7,   -3,    7,   -3,   -3,   -3,    7,   -3,   -3,
          -3,   -3,   -3,    7,   -3,   -3,   -3,    7,   -3,    7,   -3,
          -3,   -3,   -3,   -3,   -3],
       [1000, 1000,   -3,   -3,   -3,   -3,   -3,    7,   -3,    7,   -3,
          -3,   -3,   -3,   -3,   -3,   -3,   -3,   -3,    7,   -3,   -3,
          -3,   -3,   -3,    7,   

In [47]:
# degree of freedom
def flip_bit(patterns, eta):
    """flip n-th bit to allow more freedom(false positive)
       if et = 0 then pattern as is
       if et = 1 then loop over each bit and force it to one
       et = 2 loop over 2 bits and flip them ... etc
       drop any duplicate patterns"""

    for nth in range(patterns.shape[1]-eta+1):
        temp = patterns.copy()
        temp[:, nth:nth+eta] = 1
        temp = np.unique(temp, axis=0)
        yield temp

f = np.zeros((5, 6), dtype=np.int8)
eta=2


[*flip_bit(f, eta)]

[array([[1, 1, 0, 0, 0, 0]], dtype=int8),
 array([[0, 1, 1, 0, 0, 0]], dtype=int8),
 array([[0, 0, 1, 1, 0, 0]], dtype=int8),
 array([[0, 0, 0, 1, 1, 0]], dtype=int8),
 array([[0, 0, 0, 0, 1, 1]], dtype=int8)]

In [48]:
np.unique(f, axis=0)

array([[0, 0, 0, 0, 0, 0]], dtype=int8)